In [ ]:
# Clona la repo e installa requisiti
!rm -rf /content/snn-ids && git clone https://github.com/devedale/snn-ids.git /content/snn-ids -q
%cd /content/snn-ids
!pip -q install -r requirements.txt kaggle snntorch scikit-learn pyyaml



# SNN su UNSW-NB15 (Colab) — download via Kaggle e baseline snntorch

Questo notebook scarica UNSW-NB15 da Kaggle (serve API `kaggle.json`), effettua preprocessing minimale e addestra una SNN semplice con snntorch.

Istruzioni Kaggle:
- Ottieni `kaggle.json` (Account → Create New API Token)
- Caricalo in Colab e posizionalo in `/root/.kaggle/kaggle.json`



In [ ]:
# Install dipendenze e setup Kaggle
!pip -q install kaggle snntorch pandas scikit-learn pyyaml
import os, json
os.makedirs('/root/.kaggle', exist_ok=True)
# Se hai caricato kaggle.json nel tuo workspace Colab, copia il path qui sotto
kaggle_src = '/content/kaggle.json'  # cambia se necessario
if os.path.exists(kaggle_src):
  !cp -f /content/kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json || true



In [ ]:
# Download UNSW-NB15 da Kaggle (sostituisci lo slug se differente)
from pathlib import Path
Path('/content/data').mkdir(exist_ok=True)
!kaggle datasets download -d mOhammad/unsw-nb15 -p /content/data -q
!unzip -q -o /content/data/*.zip -d /content/data
!ls -lh /content/data | head -n 20



In [ ]:
# Caricamento e preprocessing minimo (adatta ai file estratti)
import pandas as pd, numpy as np
from glob import glob
files = sorted(glob('/content/data/**/*.csv', recursive=True)) or sorted(glob('/content/data/*.csv'))
print(len(files), 'csv trovati')

# Esempio: concatena tutti i CSV disponibili (alcuni release hanno train/test separati)
dfs = []
for f in files[:5]:  # limita per demo se necessario
  try:
    df = pd.read_csv(f)
    dfs.append(df)
  except Exception as e:
    print('skip', f, e)

df = pd.concat(dfs, ignore_index=True)
print(df.shape)

# Colonna target (adatta se differente nella tua versione)
label_col = 'label' if 'label' in df.columns else ('Label' if 'Label' in df.columns else None)
assert label_col is not None, 'Impossibile trovare colonna etichetta (label/Label)'

# Binarizza: normal/benign=0, attack/altri=1
labels = df[label_col].astype(str).str.lower()
y = (~labels.str.contains('normal') & ~labels.str.contains('benign')).astype(int).values

# Selezione feature numeriche + one-hot su poche categoriali
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_candidates = [c for c in df.columns if df[c].dtype == object and c != label_col]
cat_cols = cat_candidates[:3]  # poche per demo

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_num = scaler.fit_transform(df[num_cols].fillna(0).values.astype(float)) if num_cols else np.zeros((len(df),0))
X_cat = pd.get_dummies(df[cat_cols].fillna('NA').astype(str)).values if cat_cols else np.zeros((len(df),0))
X = np.hstack([X_num, X_cat]).astype(np.float32)

# Train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
input_size, output_size = X_train.shape[1], 2
X_train.shape, X_test.shape



In [ ]:
# SNN snntorch (rate coding) — identico al notebook NSL-KDD
import torch, torch.nn as nn, snntorch as snn
from torch.utils.data import TensorDataset, DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
timesteps = 10

def rate_encode(X, timesteps=10):
  Xc = torch.tensor(X, device=device)
  return Xc.unsqueeze(1).repeat(1, timesteps, 1)

train_loader = DataLoader(TensorDataset(rate_encode(X_train,timesteps), torch.tensor(y_train,device=device)), batch_size=256, shuffle=True)
test_loader  = DataLoader(TensorDataset(rate_encode(X_test, timesteps), torch.tensor(y_test, device=device)), batch_size=512, shuffle=False)

class SNNNet(nn.Module):
  def __init__(self, input_size, hidden=256, output_size=2):
    super().__init__()
    self.fc1 = nn.Linear(input_size, hidden)
    self.lif1 = snn.Leaky(beta=0.9)
    self.fc2 = nn.Linear(hidden, output_size)
    self.lif2 = snn.Leaky(beta=0.9)
  def forward(self, x):
    mem1 = self.lif1.init_leaky(); mem2 = self.lif2.init_leaky()
    spk_sum = 0
    for t in range(x.size(1)):
      cur = self.fc1(x[:, t, :])
      spk1, mem1 = self.lif1(cur, mem1)
      cur2 = self.fc2(spk1)
      spk2, mem2 = self.lif2(cur2, mem2)
      spk_sum = spk_sum + spk2
    return spk_sum

model = SNNNet(input_size, 256, output_size).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()



In [ ]:
# Training + valutazione + salvataggio
import numpy as np

def accuracy(logits, y):
  preds = torch.argmax(logits, dim=1)
  return (preds == y).float().mean().item()

for epoch in range(5):
  model.train(); losses=[]; accs=[]
  for Xb, yb in train_loader:
    opt.zero_grad(); logits = model(Xb)
    loss = criterion(logits, yb)
    loss.backward(); opt.step()
    losses.append(loss.item()); accs.append(accuracy(logits, yb))
  print(f"Epoch {epoch+1}: loss={np.mean(losses):.4f}, acc={np.mean(accs):.3f}")

model.eval(); accs=[]
with torch.no_grad():
  for Xb, yb in test_loader:
    logits = model(Xb)
    accs.append(accuracy(logits, yb))
print(f"Test Accuracy: {np.mean(accs):.3f}")

from pathlib import Path
Path('/content/out').mkdir(exist_ok=True)
torch.save(model.state_dict(), "/content/out/snn_unsw_nb15.pt")
!ls -lh /content/out

